In [ ]:
%pip install "pyautogen[teachable]"
%pip install pysqlite3-binary

In [ ]:
# Hack to use newer sqlite3 version for chromadb compatibility

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


from autogen.agentchat.contrib.skilled_agent import SkilledAgent
from autogen import UserProxyAgent
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-1106"],
    },
)

llm_config = {
    "config_list": config_list,
    "timeout": 60,
    "cache_seed": None,  # Use an int to seed the response cache. Use None to disable caching.
}

teach_config={
    "verbosity": 2,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    "reset_db": False,  # Set to True to start over with an empty database.
    "path_to_db_dir": "./tmp/notebook/teachable_agent_db",  # Path to the directory where the database will be stored.
    "recall_threshold": 1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
}

    
skilled_agent = SkilledAgent(
    name="skilledAGENT",
    llm_config=llm_config,
    memory_config=teach_config)

user_proxy = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=3,
)

In [ ]:
# Perform skill
task = "get all the cards from a given MTG set (example KTK), using the Scryfall API"
skill_prompt=f"""Write python code to {task}. Confirm the code is working and produces the correct output"""
user_proxy.initiate_chat(
    skilled_agent,
    message=skill_prompt,
)

In [ ]:
# Reflect and refactor skill - needed?
task4 = """Reflect on the sequence and refactor a into well-documented, generalized python function
to perform similar tasks for coding steps in future. Don't include any code outside of the function definition"""
user_proxy.initiate_chat(skilled_agent, message=task4, clear_history=False) 


In [ ]:
skill = skilled_agent.generated_functions[-1]
print(f"Learned the following skill:\n{skill}")
skilled_agent.remember_latest_function()

In [ ]:
task = "print all the cards from the MTG set Strixhaven, only the ones that start with G"
skill_prompt = f"Use provided functions to {task}"
skilled_agent.use_skills = True
user_proxy.initiate_chat(skilled_agent, message=skill_prompt, clear_history=True)
skilled_agent.use_skills = False


In [ ]:
#skilled_agent.function_store.get_relevant_functions('get cards from a set M21')
skilled_agent.function_store.vec_db.peek()